# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY2')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [6]:
# !pip install langchain_community

In [8]:
from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains import LLMChain
from langchain_classic.chains.llm import LLMChain

In [ ]:
#Replace None by your own value and justify
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0)

# I chose 0 because I want to see how the model would deterministically work. And gpt 3.5 because it is good enough.


In [14]:
# df_text = df.head().to_string(index=False)
df_text = df.head(20).to_markdown(index=False)

prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
 ''' 
 You are given a markdown table containing 2 columns `Product` and `Review`:
 
 {df_text}

 ''' 
)

In [15]:

chain = LLMChain(llm=llm, prompt=prompt)

In [17]:
product = 'Describe the first product using information from both columns.'#Select a product type to be describe
chain.run(product)

'| Product | Review |\n|---------|--------|\n| Laptop  | Great performance and long battery life. |\n\nThe first product is a laptop that has great performance and long battery life according to the review.'

## SimpleSequentialChain

In [28]:
# from langchain.chains import SimpleSequentialChain
from langchain_classic.chains import SimpleSequentialChain

In [36]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
 ''' 
 You are given a markdown table containing 2 columns `Product` and `Review`:
 
 {df_text}
 
 Take ONLY the first row's value from the "Product" column and write a 1-2 sentence product description.

 ''' 
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [39]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    '''
    Rewrite the product description into 3 bullet points. 

    {text}
    '''
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [40]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [41]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Product: iPhone 12

The iPhone 12 is a sleek and powerful smartphone with an impressive camera and fast performance.
- Sleek and powerful smartphone design
- Impressive camera quality
- Fast performance for seamless use of apps and features

> Finished chain.


'- Sleek and powerful smartphone design\n- Impressive camera quality\n- Fast performance for seamless use of apps and features'

**Repeat the above twice for different products**

In [ ]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
 ''' 
 You are given a markdown table containing 2 columns `Product` and `Review`:
 
 {df_text}
 
 Take ONLY the LAST row's value from the "Product" column and write a 1-2 sentence product description.

 ''' 
)

second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    '''
    Rewrite the product description into 3 bullet points. 

    {text}
    '''
)


chain_one2 = LLMChain(llm=llm, prompt=first_prompt)
chain_two2 = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain2 = SimpleSequentialChain(chains=[chain_one2, chain_two2],
                                             verbose=True
                                            )

overall_simple_chain2.run(product)

In [ ]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    #Repeat the initial query or create a new query that would feed into the second prompt
 ''' 
 You are given a markdown table containing 2 columns `Product` and `Review`:
 
 {df_text}
 
 Take ONLY the Second row's value from the "Product" column and write a 1-2 sentence product description.

 ''' 
)

second_prompt = ChatPromptTemplate.from_template(
    #Write the second prompt query that takes an input variable whose input will come from the previous prompt"
    '''
    Rewrite the product description into 3 bullet points. 

    {text}
    '''
)


chain_one3 = LLMChain(llm=llm, prompt=first_prompt)
chain_two3 = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain3 = SimpleSequentialChain(chains=[chain_one3, chain_two3],
                                             verbose=True
                                            )

overall_simple_chain3.run(product)

## SequentialChain

In [56]:
# from langchain.chains import SequentialChain
from langchain_classic.chains import SimpleSequentialChain
from langchain_classic.chains import SequentialChain

In [27]:
df['Review'][0]

'I ordered a king size set. My only criticism would be that I wish seller would offer the king size set with 4 pillowcases. I separately ordered a two pack of pillowcases so I could have a total of four. When I saw the two packages, it looked like the color did not exactly match. Customer service was excellent about sending me two more pillowcases so I would have four that matched. Excellent! For the cost of these sheets, I am satisfied with the characteristics and coolness of the sheets.'

In [63]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  ''' 
  You are given a markdown table containing 2 columns `Product` and `Review`
  
  {df_text}
  
  Your task is to translate from English to Portuguese the review of the first product.  
  '''
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='translation_key' #Give a name to your output
                    )


In [64]:
second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    '''
    Now summarize the review from the text
    
    {translation_key}
    '''
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='summarize_key' #give a name to this output
                    )


In [65]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    '''
    Now translate the review from Portuguese back to english from the text
    
    {summarize_key}
    '''
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='translation_key3'
                      )


In [66]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        '''
        Now compare the original review from {df_text} and the translation {translation_key3}
        '''
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='comparison_key'
                     )


In [67]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=['df_text'],
    output_variables=['translation_key', 'summarize_key', 'translation_key3','comparison_key'],
    verbose=True
)

In [68]:
review = df.Review[5]
overall_chain(review)

C:\Users\felip\AppData\Local\Temp\ipykernel_28656\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'df_text': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translation_key': 'Eu acho o sabor medíocre. A espuma não dura, é estranho. Compro os mesmos no mercado e o sabor é muito melhor... Lote antigo ou falsificação!?',
 'summarize_key': 'O revisor achou o sabor do produto medíocre e notou que a espuma não durava, o que considerou estranho. Comparou com produtos similares comprados no mercado e considerou que o sabor destes era muito melhor. Suspeita que o produto avaliado possa ser um lote antigo ou falsificado.',
 'translation_key3': 'The reviewer found the taste of the product mediocre and noticed that the foam did not last, which they considered strange. They compared it to similar products bought in the market and considered that the taste of those was much better. They suspect that the evaluated product may be an old or counterfeit batch.',
 'comparison_k

**Repeat the above twice for different products or reviews**

In [ ]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  ''' 
  You are given a markdown table containing 2 columns `Product` and `Review`
  
  {df_text}
  
  Your task is to translate from English to Portuguese the review of the second product.  
  '''
)

second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    '''
    Now summarize the review from the text
    
    {translation_key}
    '''
)

third_prompt = ChatPromptTemplate.from_template(
    '''
    Now translate the review from Portuguese back to english from the text
    
    {summarize_key}
    '''
)

fourth_prompt = ChatPromptTemplate.from_template(
        '''
        Now compare the original review from {df_text} and the translation {translation_key3}
        '''
)

chain_one4 = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='translation_key' #Give a name to your output
                    )

chain_two4 = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='summarize_key' #give a name to this output
                    )

chain_three4 = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='translation_key3'
                      )

chain_four4 = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='comparison_key'
                     )

overall_chain = SequentialChain(
    chains=[chain_one4, chain_two4, chain_three4, chain_four4],
    input_variables=['df_text'],
    output_variables=['translation_key', 'summarize_key', 'translation_key3','comparison_key'],
    verbose=True
)

In [ ]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
  #This prompt should translate a review
  ''' 
  You are given a markdown table containing 2 columns `Product` and `Review`
  
  {df_text}
  
  Your task is to translate from English to Portuguese the review of the LAST product.  
  '''
)

second_prompt = ChatPromptTemplate.from_template(
    #Write a promplt to summarize a review
    '''
    Now summarize the review from the text
    
    {translation_key}
    '''
)

third_prompt = ChatPromptTemplate.from_template(
    '''
    Now translate the review from Portuguese back to english from the text
    
    {summarize_key}
    '''
)

fourth_prompt = ChatPromptTemplate.from_template(
        '''
        Now compare the original review from {df_text} and the translation {translation_key3}
        '''
)

chain_one5 = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='translation_key' #Give a name to your output
                    )

chain_two5 = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='summarize_key' #give a name to this output
                    )

chain_three5 = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='translation_key3'
                      )

chain_four5 = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='comparison_key'
                     )

overall_chain = SequentialChain(
    chains=[chain_one5, chain_two5, chain_three5, chain_four5],
    input_variables=['df_text'],
    output_variables=['translation_key', 'summarize_key', 'translation_key3','comparison_key'],
    verbose=True
)

## Router Chain

In [69]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [70]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [74]:
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain_classic.prompts import PromptTemplate

In [75]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model='gpt-3.5-turbo', temperature=0)

In [76]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [77]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [78]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [79]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [80]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\felip\AppData\Local\Temp\ipykernel_28656\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [81]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [82]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [83]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [ ]:
chain.run("What is the photo electric effect?")

In [ ]:
chain.run("How many letters there are in the word Park?")

In [ ]:
chain.run("What is the scientific name for the human species, be deatiled?")